In [1]:
# @title Importing modules
from CustomDataGen import CustomDataGen
import Constants
import Util
import Plot
import Preprocess
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import Model
import pandas as pd
import tensorflow as tf


2022-12-10 17:27:35.116968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 17:27:35.833231: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-10 17:27:35.833295: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-10 17:27:35.833301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# @title Tensor flow gpu check
tf.test.gpu_device_name()

2022-12-10 17:27:47.909969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 17:27:47.925513: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 17:27:47.944231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 17:27:47.944555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

'/device:GPU:0'

In [4]:
tf.random.set_seed(0)

In [5]:
# @title Setting up download path
DATA_ROOT = "./"
DATASET_NAME = "bsd"
ORIGIN_URL = "http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz"
DOWNLOAD_DIR = DATA_ROOT + "dataset" + "/" + DATASET_NAME

In [6]:
DOWNLOAD_DIR

'./dataset/bsd'

In [12]:
# @title Downloading flickr8k dataset
Util.download_dataset(ORIGIN_URL, DOWNLOAD_DIR)

Downloaded successfully in /tmp/.keras/./dataset/flickr8k/Flickr8k_Dataset.zip


In [13]:
# @title Creating directories for pre-processing images 
CLEAN_IMG_SUB_DIR = "/Flicker8k_Dataset"
CLEAN_IMG_DIR = DOWNLOAD_DIR + CLEAN_IMG_SUB_DIR
NOISE_DIR = DOWNLOAD_DIR + "/noise"

In [14]:
Util.make_dir(NOISE_DIR)
NOISE_DIR = NOISE_DIR + \
    f"/m_{Constants.MEAN}__v_{Constants.MEAN}".replace(".", "d")
Util.make_dir(NOISE_DIR)
NOISE_1_DIR = NOISE_DIR + "/noise_1"
Util.make_dir(NOISE_1_DIR)
NOISE_2_DIR = NOISE_DIR + "/noise_2"
Util.make_dir(NOISE_2_DIR)

 ./dataset/flickr8k/noise already exists
 ./dataset/flickr8k/noise/m_0__v_0 already exists
 ./dataset/flickr8k/noise/m_0__v_0/noise_1 already exists
 ./dataset/flickr8k/noise/m_0__v_0/noise_2 already exists


In [ ]:
# @title Pre-processing images 
Preprocess.preprocess_images(CLEAN_IMG_DIR, NOISE_1_DIR, NOISE_2_DIR)

In [ ]:
# @title Plot first n images from clean and noise dirs 
n = 4  # @param {type:"slider", min:1, max:15, step:1}
Plot.plot_first_n(n, CLEAN_IMG_DIR, NOISE_1_DIR, NOISE_2_DIR)

In [ ]:
noise_1_paths = pd.Series(Util.get_noise_img_path(
    CLEAN_IMG_DIR, NOISE_1_DIR), name='noise_1').astype(str)

noise_1_paths.describe()

In [ ]:
noise_2_paths = pd.Series(Util.get_noise_img_path(
    CLEAN_IMG_DIR, NOISE_2_DIR), name='noise_2').astype(str)

noise_2_paths.describe()

In [ ]:
images_df = pd.concat([noise_1_paths, noise_2_paths], axis=1).sample(
    frac=1.0, random_state=1).reset_index(drop=True)
images_df.describe()

In [ ]:
train_size = 0.70  # @param {type:"slider", min:0, max:1, step:0.01}
train_df, test_df = train_test_split(
    images_df, train_size=train_size, shuffle=True, random_state=1)

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
traingen = CustomDataGen(train_df)
print(f"Number of train batches {len(traingen)}")

valgen = CustomDataGen(test_df)
print(f"Number of validation batches {len(valgen)}")

In [ ]:
autoencoder = Model.UNet()
autoencoder_optimizer = tf.keras.optimizers.Adam(Constants.LEARNING_RATE, beta_1=Constants.BETA_1)
autoencoder.compile(
    optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=["accuracy"])

In [ ]:
VERSION = "unet_1"  # @param {type:"string"}

In [ ]:
checkpoint_path = "my_model/gauss" + \
    f"/m_{Constants.MEAN}__v_{Constants.VAR}".replace(
        ".", "d") + f"_version_{VERSION}"
monitor = "val_accuracy"
mode = "max"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor=monitor,
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode=mode,
)

In [ ]:
# @title Train the model
EPOCHS = 15
history = autoencoder.fit(traingen,
                          validation_data=valgen,
                          epochs=EPOCHS,
                          shuffle=True,
                          callbacks=[model_checkpoint])

In [ ]:
Plot.plot_history(history)

In [ ]:
FILE_NAME = "./models/" + f"/m_{Constants.MEAN}__v_{Constants.VAR}".replace(
    ".", "d") + f"_version_{VERSION}" + DATASET_NAME + ".h"

In [ ]:
TF_LITE_NAME = "models_tf_lite/" + \
    f"/m_{Constants.MEAN}__v_{Constants.VAR}".replace(".", "d") + \
    f"_version_{VERSION}" + DATASET_NAME + ".h"

In [ ]:
Util.save_model(FILE_NAME, autoencoder)

In [ ]:
Util.save_tf_lite(FILE_NAME, autoencoder)

In [ ]:
for noise_img_path in test_df["noise_1"][:5]:
    Plot.vsualize_predictions(autoencoder,noise_img_path,CLEAN_IMG_DIR)